In [1]:
from dotenv import load_dotenv

load_dotenv()

True

랭그래프를 구축하고 대화하는 과정에서 사람이 직접 개입할 수 있도록 설정

특정 액션을 취하기 이전에 멈추도록 설정

In [1]:
# 웹검색 에이전트를 만들었을 때 코드 그대로 사용
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph

from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

class State(TypedDict):
    messages: Annotated[list, add_messages]

tool = TavilySearchResults(max_results=2)
tools = [tool]
tool_node = ToolNode(tools)

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    result = llm_with_tools.invoke(state["messages"])
    return {"messages": [result]}

graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)

graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile(
    checkpointer=memory,
    interrupt_before=["tools"]) # 라이브러리 사용없이 interrupt_before 파라미터만 추가해서 사용하면 된다.
                                # 특정 노드를 실행하기 이전에 멈추게 하는 파라미터 (["toools"]를 넣어줬을때 toolsa노드를 실행하기 이전에 그래프가 실행을 멈춤)

In [ ]:
user_input = "Langgraph가 뭐야?" # 질문 
config = {"configurable": {"thread_id": "1"}} # thread_id: 1 에 저장

# 대화를 스트리밍 하도록 설정
events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)

for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

# Langgraph가 뭐야? 라는 질문을 주면 웹검색이 필요할 건데, 
# 아래 결과를 보면 실제로 웹 검색까지 실행하는 (Tool Calls) AI Message까지 볼 수 있다.
# 이 다음 ToolNode로 들어가서 Tool이 실행되는 것 까지는 확인할 수 없다.
# interrupt_before 라는 파라미터를 설정해줬기 때문이다.

# 어떤 노드가 실행되기 이전에 한번 멈추고 중간에 사용자가 입력을 하도록 만들어 줄 수 있다.
# 이전에 있었던 스테이트 값들을 변경을 해서 내가 원하는 형태로 대화를 이끌어갈 수 있게끔 만들어줄 수도 있다.

================================ Human Message =================================

Langgraph가 뭐야?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_y4tzCwuasYnY08vbKAFTKiZS)
 Call ID: call_y4tzCwuasYnY08vbKAFTKiZS
  Args:
    query: Langgraph
